<a href="https://colab.research.google.com/github/pank8/python/blob/master/MovieLens_100K_Dataset_(spark).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузить данные в spark

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [0]:
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

In [0]:
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
from pyspark import SparkFiles


In [0]:
from pyspark import SparkContext

In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [0]:
f1 = sc.textFile("u.data")
data_rdd = f1.map(lambda line: [x for x in line.split("\t")])

In [97]:
data_rdd.take(1)

[['196', '242', '3', '881250949']]

In [0]:
data = data_rdd.toDF(["userid", "movieid", "rating", "timestamp"])

In [116]:
data.show(5)

+------+-------+------+---------+
|userid|movieid|rating|timestamp|
+------+-------+------+---------+
|   196|    242|     3|881250949|
|   186|    302|     3|891717742|
|    22|    377|     1|878887116|
|   244|     51|     2|880606923|
|   166|    346|     1|886397596|
+------+-------+------+---------+
only showing top 5 rows



In [0]:
f2 = sc.textFile("u.genre")
genre_rdd = f2.map(lambda line: [x for x in line.split("|")])

In [104]:
genre_rdd.take(1)

[['unknown', '0']]

In [0]:
genre = genre_rdd.toDF(["genre", "genreid"])

In [117]:
genre.show(5)

+----------+-------+
|     genre|genreid|
+----------+-------+
|   unknown|      0|
|    Action|      1|
| Adventure|      2|
| Animation|      3|
|Children's|      4|
+----------+-------+
only showing top 5 rows



In [0]:
f3 = sc.textFile('u.info')
info_rdd = f3.map(lambda line: [x for x in line.split(" ")])

In [66]:
info_rdd.take(1)

[['943', 'users']]

In [0]:
info = info_rdd.toDF(["count", "name"])

In [68]:
info.show(5)

+------+-------+
| count|   name|
+------+-------+
|   943|  users|
|  1682|  items|
|100000|ratings|
+------+-------+



In [0]:
f4 = sc.textFile('u.occupation')
occupation_rdd = f4.map(lambda line: [x for x in line.split("\t")])

In [70]:
occupation_rdd.take(1)

[['administrator']]

In [0]:
occupation = occupation_rdd.toDF(["occupation"])

In [72]:
occupation.show(5)

+-------------+
|   occupation|
+-------------+
|administrator|
|       artist|
|       doctor|
|     educator|
|     engineer|
+-------------+
only showing top 5 rows



In [0]:
f5 = sc.textFile('u.item')
item_rdd = f5.map(lambda line: [x for x in line.split("|")])

In [273]:
item_rdd.take(1)

[['1',
  'Toy Story (1995)',
  '01-Jan-1995',
  '',
  'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)',
  '0',
  '0',
  '0',
  '1',
  '1',
  '1',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0']]

In [0]:
item = item_rdd.toDF(["movieid", "Title", "release", "video release date ", "URL","unknown", "Action", "Adventure", "Animation", "Childrens","Comedy", "Crime", "Documentary", "Drama", "Fantasy","FilmNoir", "Horror", "Musical", "Mystery", "Romance","SciFi", "Thriller", "War", "Western"])

In [275]:
item.show(5)

+-------+-----------------+-----------+-------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
|movieid|            Title|    release|video release date |                 URL|unknown|Action|Adventure|Animation|Childrens|Comedy|Crime|Documentary|Drama|Fantasy|FilmNoir|Horror|Musical|Mystery|Romance|SciFi|Thriller|War|Western|
+-------+-----------------+-----------+-------------------+--------------------+-------+------+---------+---------+---------+------+-----+-----------+-----+-------+--------+------+-------+-------+-------+-----+--------+---+-------+
|      1| Toy Story (1995)|01-Jan-1995|                   |http://us.imdb.co...|      0|     0|        0|        1|        1|     1|    0|          0|    0|      0|       0|     0|      0|      0|      0|    0|       0|  0|      0|
|      2| GoldenEye (1995)|01-Jan-1995|                   |http://us.imd

In [0]:
f6 = sc.textFile('u.user')
user_rdd = f6.map(lambda line: [x for x in line.split("|")])

In [79]:
user_rdd.take(1)

[['1', '24', 'M', 'technician', '85711']]

In [0]:
user = user_rdd.toDF( ["userid", "age", "gender", "occupation", "zip code"])

In [81]:
user.show(5)

+------+---+------+----------+--------+
|userid|age|gender|occupation|zip code|
+------+---+------+----------+--------+
|     1| 24|     M|technician|   85711|
|     2| 53|     F|     other|   94043|
|     3| 23|     M|    writer|   32067|
|     4| 24|     M|technician|   43537|
|     5| 33|     F|     other|   15213|
+------+---+------+----------+--------+
only showing top 5 rows



# Средствами спарка вывести среднюю оценку для каждого фильма

In [0]:
from pyspark.sql import functions as F

In [0]:
movies = data.join(item, on='movieid').select('movieid', 'Title', "userid", "rating")

In [38]:
movies.show(5)

+-------+-------------+------+------+
|movieid|        Title|userid|rating|
+-------+-------------+------+------+
|   1090|Sliver (1993)|    56|     3|
|   1090|Sliver (1993)|   303|     1|
|   1090|Sliver (1993)|   268|     2|
|   1090|Sliver (1993)|   145|     2|
|   1090|Sliver (1993)|   320|     3|
+-------+-------------+------+------+
only showing top 5 rows



In [0]:
movies_avg = movies.groupBy('Title','movieid').agg(F.count(movies.rating).alias('count'), F.avg(movies.rating).alias('avg'))

In [179]:
movies_avg.show(5)

+--------------------+-------+-----+------------------+
|               Title|movieid|count|               avg|
+--------------------+-------+-----+------------------+
|       Sliver (1993)|   1090|   37|2.4054054054054053|
|      Stalker (1979)|   1159|   11|3.5454545454545454|
|    Mr. Jones (1993)|   1436|    2|               2.5|
|World of Apu, The...|   1512|    6|               4.0|
|Wend Kuuni (God's...|   1572|    1|               1.0|
+--------------------+-------+-----+------------------+
only showing top 5 rows



# В спарке получить 2 датафрейма с 5-ю самыми популярными и самыми непопулярными фильмами (по количеству оценок, либо по самой оценке - на Ваш выбор)

In [42]:
#5 самых популярных
movies_avg.orderBy('avg', ascending=False).show(5)

+--------------------+-----+---+
|               Title|count|avg|
+--------------------+-----+---+
|Someone Else's Am...|    1|5.0|
|Saint of Fort Was...|    2|5.0|
|     Star Kid (1997)|    3|5.0|
|Aiqing wansui (1994)|    1|5.0|
|Marlene Dietrich:...|    1|5.0|
+--------------------+-----+---+
only showing top 5 rows



In [48]:
#5 самых непопулярных
movies_avg.orderBy('avg', ascending=True).show(5)

+--------------------+-----+---+
|               Title|count|avg|
+--------------------+-----+---+
|Lashou shentan (1...|    1|1.0|
|JLG/JLG - autopor...|    1|1.0|
|Modern Affair, A ...|    1|1.0|
|Vie est belle, La...|    1|1.0|
|        Gordy (1995)|    3|1.0|
+--------------------+-----+---+
only showing top 5 rows



# Средствами спарка соедините информацию по фильмам и жанрам (u.genre)
# 1.	 Дублирование строк в данном случае - ок.
# 2.	Интересует такая структура - id фильма | название фильма | жанр

In [0]:
#подскажите, пожалуйста а есть какой-то вариант не руками названия стоблцов переписывать?
item_new = item.selectExpr('movieid','Title', "stack(19, 'unknown', unknown,'Action', Action,'Adventure', Adventure,'Animation', Animation, 'Childrens', Childrens, 'Comedy', Comedy, 'Crime', Crime, 'Documentary', Documentary, 'Drama', Drama, 'Fantasy', Fantasy, 'FilmNoir', FilmNoir, 'Horror', Horror, 'Musical', Musical, 'Mystery', Mystery, 'Romance', Romance, 'SciFi', SciFi, 'Thriller', Thriller, 'War', War, 'Western', Western) as (Genre, Flag)").where("Flag !=0")

In [277]:
item_new.show()

+-------+--------------------+---------+----+
|movieid|               Title|    Genre|Flag|
+-------+--------------------+---------+----+
|      1|    Toy Story (1995)|Animation|   1|
|      1|    Toy Story (1995)|Childrens|   1|
|      1|    Toy Story (1995)|   Comedy|   1|
|      2|    GoldenEye (1995)|   Action|   1|
|      2|    GoldenEye (1995)|Adventure|   1|
|      2|    GoldenEye (1995)| Thriller|   1|
|      3|   Four Rooms (1995)| Thriller|   1|
|      4|   Get Shorty (1995)|   Action|   1|
|      4|   Get Shorty (1995)|   Comedy|   1|
|      4|   Get Shorty (1995)|    Drama|   1|
|      5|      Copycat (1995)|    Crime|   1|
|      5|      Copycat (1995)|    Drama|   1|
|      5|      Copycat (1995)| Thriller|   1|
|      6|Shanghai Triad (Y...|    Drama|   1|
|      7|Twelve Monkeys (1...|    Drama|   1|
|      7|Twelve Monkeys (1...|    SciFi|   1|
|      8|         Babe (1995)|Childrens|   1|
|      8|         Babe (1995)|   Comedy|   1|
|      8|         Babe (1995)|    

# Посчитайте средствами спарка среднюю оценку для каждого жанра

In [0]:
item_new_with_ratings = item_new.join(movies_avg_df, on='movieid')

In [281]:
item_new_with_ratings.show(5)

+-------+----------------+--------+----+----------------+-----+------------------+
|movieid|           Title|   Genre|Flag|           Title|count|               avg|
+-------+----------------+--------+----+----------------+-----+------------------+
|   1090|   Sliver (1993)|Thriller|   1|   Sliver (1993)|   37|2.4054054054054053|
|   1159|  Stalker (1979)| Mystery|   1|  Stalker (1979)|   11|3.5454545454545454|
|   1159|  Stalker (1979)|   SciFi|   1|  Stalker (1979)|   11|3.5454545454545454|
|   1436|Mr. Jones (1993)|   Drama|   1|Mr. Jones (1993)|    2|               2.5|
|   1436|Mr. Jones (1993)| Romance|   1|Mr. Jones (1993)|    2|               2.5|
+-------+----------------+--------+----+----------------+-----+------------------+
only showing top 5 rows



In [0]:
genre_avg = item_new_with_ratings.groupBy('Genre').agg(F.avg(item_new_with_ratings.avg).alias('avg'))

In [284]:
genre_avg.show()

+-----------+------------------+
|      Genre|               avg|
+-----------+------------------+
|      Crime|3.2110147495997534|
|    Romance|   3.2440490301648|
|   Thriller| 3.136692475563834|
|  Adventure| 3.143967313697611|
|    unknown|2.2222222222222223|
|      Drama| 3.187353414297331|
|        War|3.4891854289435704|
|Documentary|3.2292730940939407|
|    Fantasy| 2.849830297920943|
|   FilmNoir|3.5483508757184237|
|    Mystery|3.3368139323830235|
|    Musical|3.3764231235398725|
|  Animation|3.2988130803823776|
|     Horror| 2.730157352996138|
|    Western| 3.185617473056144|
|     Comedy|3.0005649618406722|
|     Action|2.9663324037589875|
|      SciFi|  3.16544605325129|
|  Childrens|2.9168848918688983|
+-----------+------------------+

